In [2]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=273
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs'
TARGET_SPECIES='N'
####################################################



os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()




def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()





av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    eta0s.append(eta0)
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    
    
    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")


    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f1
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(os.getcwd()))
print("==================================")
print("""
target_atom_labels=\t{}\n
av_theta_xs=\t{}\n    
av_theta_ys=\t{}\n  
av_theta_y2s=\t{}\n
av_theta_x2s=\t{}\n 
cq_coef0s=\t{}\n
cq_coef1s=\t{}\n
eta_coefs=\t{}\n
cq0s=\t{}\n
cqprime0s=\t{}\n
cqprime1s=\t{}\n 
eta0s=\t{}\n
etaprimes=\t{}\n
fq0s=\t{}\n 
fqprimes=\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()


print("eta0s\n---------")
for item in target_atom_labels:
    print(item, eta0s[target_atom_labels.index(item)])
print()





endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()





Begin on Mon Feb 19 14:18:58 2018

Working dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs


TEMP:            273
TARGET_DIR:      /Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs
TARGET_SPECIES:  N


There are 901 efg files in this directory.

901 efg files opened without error.

The indices of N atoms are:
1 3 4 6 23 25 26 28 37 39 40 42 51 53 54 56 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
900 
Report for atomic specie:	1
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs
<tx>:	 -0.0103557186866 	<ty>:	 -0.00232026994242 
 <tx^2>	: 0.00435854504478 	<ty^2>: 0.00334955853112 

cq0=	-6.2006
eta0=	0.52707
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs
T=273K, nuclear site: 1  =
1-3/2(<tx^2>+<ty^2>)		=0.9884378446361469
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.9888366995274624

Cq0:		-6.2006
Cq' from coefficien

Fetching the angles in file efg.{}.out
900 
Report for atomic specie:	37
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs
<tx>:	 0.0226334670137 	<ty>:	 -0.00192542591444 
 <tx^2>	: 0.0417582920782 	<ty^2>: 0.0187978021375 

cq0=	-0.931
eta0=	0.47174
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927/scfs
T=273K, nuclear site: 37  =
1-3/2(<tx^2>+<ty^2>)		=0.9091658586764637
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.9172893948199059

Cq0:		-0.931
Cq' from coefficient 1-3/2(<>+<>):		-0.8464334144277877
Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):	-0.8539964265773324
Computing NQR frequencies for spin:	1
using function <function f1 at 0x113223e18>	and simple coefficient
fqs=[-0.19964825 -0.53500094 -0.73464919]



Working on atomic specie:	39
Fetching the angles in file efg.{}.out
900 
Report for atomic specie:	39
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80

40 0.917867228658
42 0.990040092837
51 0.928437185952
53 0.988357592041
54 0.909711417335
56 0.989141076329

cq_coef1s
---------
1 0.988836699527
3 0.905334306615
4 0.988310200251
6 0.927040349254
23 0.947436031887
25 0.987696389782
26 0.895110286831
28 0.991544965037
37 0.91728939482
39 0.988914432295
40 0.92282288593
42 0.990096280667
51 0.933233061006
53 0.98881261041
54 0.919348316247
56 0.989272230643

cq0s
---------
1 -6.2006
3 -0.8189
4 -6.0103
6 -0.931
23 -0.9308
25 -6.0105
26 -0.8185
28 -6.2006
37 -0.931
39 -6.0103
40 -0.8189
42 -6.2006
51 -0.9308
53 -6.0105
54 -0.8185
56 -6.2006

cqprime0s
---------
1 -6.12890769945
3 -0.735420262032
4 -5.94377700032
6 -0.85710648592
23 -0.874348565426
25 -5.93121836257
26 -0.719749565084
28 -6.15217896585
37 -0.846433414428
39 -5.94567249962
40 -0.751641473548
42 -6.13884259965
51 -0.864189332684
53 -5.94052330696
54 -0.744598795088
56 -6.13326815788

cqprime1s
---------
1 -6.13138083909
3 -0.741378263687
4 -5.94004079657
6 -0.863074565155
2

In [19]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime



target_atom_labels=	[1, 3, 4, 6, 23, 25, 26, 28, 37, 39, 40, 42, 51, 53, 54, 56]

av_theta_xs=	[-0.010355718686608378, 0.0065067112559019395, 0.016869114688317238, 0.020489678080366384, -0.016682767706222835, -0.0011859393280812351, 0.040795330680082557, 0.0052049145934683922, 0.022633467013737904, 0.017393998184580969, 0.001034366110388226, -0.011318461591954482, -0.019491241686798721, -0.0065875559630895414, -0.0027059757779704261, 0.0062512639603429562]
    
av_theta_ys=	[-0.0023202699424233826, -0.0094447767187146609, -0.013428875111828949, -0.0033288729728474954, 0.0079516693063739485, -0.0060329739186572974, -0.015762161997247254, -0.00062881344485362419, -0.0019254259144444053, -0.012522382076493832, -0.017719739758701048, 0.0013432965451751048, -0.0018065302864361523, 0.00021141553492396443, -0.017238860762116767, 0.0022520881024812103]
  
av_theta_y2s=	[0.0033495585311216253, 0.024244761764722695, 0.0045309237517600597, 0.017397459712355991, 0.0087975556916394387, 0.0031961605269510002, 0.019132365804965534, 0.0034200575515081788, 0.018797802137533771, 0.0040347593083192475, 0.020746307425059712, 0.0032488996450076623, 0.017080436527603129, 0.0032648100052670458, 0.017202641851934521, 0.0034537501474861884]

av_theta_x2s=	[0.0043585450447804598, 0.043716110924671592, 0.0028478537560698079, 0.035515905185634022, 0.031634642112329109, 0.005597521355086359, 0.061299672407772232, 0.0017860014486808817, 0.041758292078157129, 0.0031337680953056916, 0.034008873469301634, 0.0033910384636135936, 0.030628106170979049, 0.0044967953005381782, 0.042989746591729966, 0.0037855323001695076]
 
cq_coef0s=	[0.98843784463614692, 0.89805869096590851, 0.98893183373825522, 0.92062995265301495, 0.93935170329404716, 0.98680947717694401, 0.87935194268089334, 0.99219091149971639, 0.90916585867646371, 0.98924720889456264, 0.91786722865845793, 0.99004009283706806, 0.92843718595212676, 0.98835759204129214, 0.90971141733450323, 0.98914107632851644]

cq_coef1s=	[0.98883669952746245, 0.9053343066146422, 0.98831020025069694, 0.92704034925368806, 0.94743603188697134, 0.98769638978180541, 0.89511028683142824, 0.99154496503712852, 0.9172893948199059, 0.9889144322949921, 0.92282288593013428, 0.99009628066741007, 0.93323306100588177, 0.98881261041028434, 0.91934831624681224, 0.98927223064292757]

eta_coefs=	[]

cq0s=	[-6.2006, -0.8189, -6.0103, -0.931, -0.9308, -6.0105, -0.8185, -6.2006, -0.931, -6.0103, -0.8189, -6.2006, -0.9308, -6.0105, -0.8185, -6.2006]

cqprime0s=	[-6.1289076994508926, -0.73542026203198241, -5.9437770003170352, -0.85710648591995697, -0.87434856542609907, -5.9312183625720225, -0.7197495650843112, -6.1521789658451409, -0.84643341442778774, -5.9456724996189898, -0.75164147354841115, -6.1388425996455238, -0.86418933268423959, -5.9405233069641872, -0.74459879508829085, -6.1332681578825987]

cqprime1s=	[-6.1313808390899833, -0.74137826368673043, -5.9400407965667634, -0.86307456515518366, -0.88187345848039289, -5.9365491507835415, -0.73264776977152402, -6.1481737102092184, -0.85399642657733243, -5.9436724124225906, -0.75569966128818689, -6.1391909979063426, -0.86865333318427473, -5.9432581948710146, -0.75248659684801578, -6.1340813933245366]
 
eta0s=	[0.52707, 0.49821, 0.49246, 0.47174, 0.472, 0.49245, 0.49828, 0.52707, 0.47174, 0.49246, 0.49821, 0.52707, 0.472, 0.49245, 0.49828, 0.52707]

etaprimes=	[]

fq0s=   [np.array([-1.61518169, -3.78908993, -5.40427162]), np.array([-0.18319686, -0.45996676, -0.64316363]), np.array([-1.46353621, -3.72606464, -5.18960086]), np.array([-0.20216571, -0.54174701, -0.74391272]),np.array([-0.20634626, -0.55258829, -0.75893455]),np.array([-1.46041424, -3.71820665, -5.17862089]),np.array([-0.17931841, -0.45015297, -0.62947138]),np.array([-1.62131448, -3.80347698, -5.42479147]),np.array([-0.19964825, -0.53500094, -0.73464919]),np.array([-1.46400294, -3.7272529 , -5.19125584]),np.array([-0.18723765, -0.47011228, -0.65734993]),np.array([-1.61779988, -3.79523201, -5.41303189]),np.array([-0.20394868, -0.54616766, -0.75011634]),np.array([-1.46270535, -3.7240398 , -5.18674516]),np.array([-0.18550934, -0.46569442, -0.65120377]),np.array([-1.61633082, -3.79178571, -5.40811653])]
 
fqprimes=	[]

TEMP=273
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927'
os.chdir(TARGET_DIR)
os.getcwd()
x=md.Md('md.in','md.out')
vol=x.cell_volume
print(TARGET_DIR,"\n","-"*len(TARGET_DIR),"\n")

def get_inds(target, inds):
    result = []
    for ind in inds:
        result.append(target[ind])
    return result

axial_inds = [ 0,  7, 11, 15]
equit_inds = [ 2,  5,  9, 13]
ax         = axial_inds
eq         = equit_inds

axial_cq0s = get_inds(cq0s, ax)
axial_eta0s= get_inds(eta0s, ax)
axial_k0s  = get_inds(cq_coef0s, ax)
axial_k1s  = get_inds(cq_coef1s, ax)
axial_cq   = np.average(get_inds(cq0s, ax))
axial_eta  = np.average(get_inds(eta0s, ax))
axial_k0   = np.average(get_inds(cq_coef0s, ax))
axial_k1   = np.average(get_inds(cq_coef1s, ax))


equit_cq0s = get_inds(cq0s, eq)
equit_eta0s= get_inds(eta0s, eq)
equit_k0s  = get_inds(cq_coef0s, eq)
equit_k1s  = get_inds(cq_coef1s, eq)
equit_cq   = np.average(get_inds(cq0s, eq))
equit_eta  = np.average(get_inds(eta0s, eq))
equit_k0   = np.average(get_inds(cq_coef0s, eq))
equit_k1   = np.average(get_inds(cq_coef1s, eq))



print("cell volume:", vol,"\n")
print("AXIAL","\n","-"*len(TARGET_DIR))
print("avg Cq0:", np.average(axial_cq0s),"\t",axial_cq)
print("avg eta0:", np.average(axial_eta0s),"\t",axial_eta)
print("avg coef0:",np.average(axial_k0s),"\t",axial_k0)
print("avg coef1:",np.average(axial_k1s),"\t",axial_k1)
print("using coef0:",axial_k0*axial_cq)
print("using coef1:",axial_k1*axial_cq)


ffunc=nqr.f1
fraw=ffunc(axial_cq,axial_eta)
f0=ffunc(axial_k0*axial_cq,axial_eta)
f1=ffunc(axial_k1*axial_cq,axial_eta)
exptcq=6.029
expteta=0.5205
exptfq=[5.306,       3.737]
print("expt cq:\t", exptcq)
print("raw freq:\t",list(reversed(fraw))[:-1])

print("expt fq:\t", exptfq)
print("adjusted f0s:\t",list(reversed(f0))[:-1])
print("adjusted f1s:\t",list(reversed(f1))[:-1])
print("\n\n")

print("EQUITORIAL","\n","-"*len(TARGET_DIR))
print("avg Cq0:", np.average(equit_cq0s),"\t",equit_cq)
print("avg eta0:", np.average(equit_eta0s),"\t",equit_eta)
print("avg coef0:",np.average(equit_k0s),"\t",equit_k0)
print("avg coef1:",np.average(equit_k1s),"\t",equit_k1)
print("using coef0:",equit_k0*equit_cq)
print("using coef1:",equit_k1*equit_cq)


ffunc=nqr.f1
fraw=ffunc(equit_cq,equit_eta)
f0=ffunc(equit_k0*equit_cq,equit_eta)
f1=ffunc(equit_k1*equit_cq,equit_eta)
exptcq=5.795
expteta=0.4980
exptfq=[5.068,       3.625]
print("expt cq:\t", exptcq)
print("raw freq:\t",list(reversed(fraw))[:-1])

print("expt fq:\t", exptfq)
print("adjusted f0s:\t",list(reversed(f0))[:-1])
print("adjusted f1s:\t",list(reversed(f1))[:-1])

/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792927 
 -------------------------------------------------------- 

cell volume: 514.202393335 

AXIAL 
 --------------------------------------------------------
avg Cq0: -6.2006 	 -6.2006
avg eta0: 0.52707 	 0.52707
avg coef0: 0.989952481325 	 0.989952481325
avg coef1: 0.989937543969 	 0.989937543969
using coef0: -6.13829935571
using coef1: -6.13820673513
expt cq:	 6.029
raw freq:	 [-5.4674875604999986, -3.8334124394999995]
expt fq:	 [5.306, 3.737]
adjusted f0s:	 [-5.4125528771325238, -3.7948961564265336]
adjusted f1s:	 [-5.412471207320964, -3.7948388953778158]



EQUITORIAL 
 --------------------------------------------------------
avg Cq0: -6.0104 	 -6.0104
avg eta0: 0.492455 	 0.492455
avg coef0: 0.988336527963 	 0.988336527963
avg coef1: 0.988433408184 	 0.988433408184
using coef0: -5.94029786767
using coef1: -5.94088015655
expt cq:	 5.795
raw freq:	 [-5.2477628830000009, -3.7678371170000005]
expt fq:	 [5.068, 3.625]
adjusted f0s:	

[3, 2]